Imports

In [4]:
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd
import time
import warnings; warnings.filterwarnings('ignore');
from tensorflow.python.client import device_lib

Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/deep-air/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [8]:
# CuDNN-powered LSTM
from keras.layers import CuDNNLSTM

In [9]:
def get_available_gpus():
    local_devices = device_lib.list_local_devices()
    return [x for x in local_devices if x.device_type == 'GPU']

In [10]:
get_available_gpus()

[name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 357957632
 locality {
   bus_id: 1
 }
 incarnation: 1033613650667949710
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:1e.0, compute capability: 3.7"]

## Baseline LSTM + room to change network architecture

In [364]:
def handle_wind_dir(data):
    '''
    Different ways to handle the string format:
     1. Drop it
     2. LabelEncode it
     3. One-hot encode it
     
    The reason I need to handle it is because Keras' pad_sequences function takes int() of
    all the columns.
    '''
    # I'll drop it first
    data = data.drop('wnd_dir', axis=1, inplace=False)
    return data

In [365]:
df = pd.read_csv('data/pollution.csv', header=0, index_col=0)

In [366]:
# Optionally drop wind dir
df = handle_wind_dir(df)

In [367]:
df.head()

pollution  dew  temp   press  wnd_spd  snow  rain
24      129.0  -16  -4.0  1020.0     1.79     0     0
25      148.0  -15  -4.0  1020.0     2.68     0     0
26      159.0  -11  -5.0  1021.0     3.57     0     0
27      181.0   -7  -5.0  1022.0     5.36     1     0
28      138.0   -7  -5.0  1022.0     6.25     2     0

In [368]:
df_features = df.iloc[:, 1:]

In [369]:
df_label = pd.DataFrame(df.iloc[:,0], columns=['pollution'])

In [370]:
def standardize(df):
    return ((df.values - np.mean(df.values, axis = 0)) / np.std(df.values, axis = 0))

In [371]:
stand = pd.DataFrame(standardize(df_features), columns=df_features.columns)

In [372]:
stand.head()

dew      temp     press   wnd_spd      snow      rain
0 -1.235589 -1.349849  0.345886 -0.441890 -0.069372 -0.137706
1 -1.166285 -1.349849  0.345886 -0.424098 -0.069372 -0.137706
2 -0.889069 -1.431862  0.443244 -0.406306 -0.069372 -0.137706
3 -0.611852 -1.431862  0.540603 -0.370522  1.245425 -0.137706
4 -0.611852 -1.431862  0.540603 -0.352730  2.560223 -0.137706

### Split Train and Leave Out set

In [373]:
test_size = 0.20
row = 1 - round(test_size*len(df_features))
train_features = df_features.iloc[:row, :]
train_label = df_label.iloc[:row, :]
leave_out_features = df_features.iloc[row:, :]
leave_out_label = df_label.iloc[row:, :]

In [374]:
def prepare_sequences(df_features, df_label):
    ''' 
    Let df = train and prepare sequences.
    '''
    input_cols = list(df_features.columns)
#     df_features['single_input_vector'] = df_features[input_cols].apply(tuple, axis=1).apply(list)\
#         .apply(lambda x: [list(x)])
    df_features['single_input_vector'] = df_features[input_cols].apply(tuple, axis=1).apply(list)\
        .apply(lambda x: [x])
    df_label['single_output_vector'] = df_label['pollution'].apply(lambda x: [[x]])
    train_full = pd.concat((df_features, df_label), axis=1)
    return df_features, df_label, train_full

In [375]:
train_features, train_label, train_full = prepare_sequences(train_features, train_label)

In [376]:
# train_features, test_features, train_label, test_label = train_test_split(stand, df_label,
#                                                                           test_size=.50, 
#                                                                           random_state=789)

In [377]:
len(train_features)

35041

In [378]:
len(train_label)

35041

In [379]:
train_features.head()

dew  temp   press  wnd_spd  snow  rain  \
24  -16  -4.0  1020.0     1.79     0     0   
25  -15  -4.0  1020.0     2.68     0     0   
26  -11  -5.0  1021.0     3.57     0     0   
27   -7  -5.0  1022.0     5.36     1     0   
28   -7  -5.0  1022.0     6.25     2     0   

                        single_input_vector  
24  [[-16.0, -4.0, 1020.0, 1.79, 0.0, 0.0]]  
25  [[-15.0, -4.0, 1020.0, 2.68, 0.0, 0.0]]  
26  [[-11.0, -5.0, 1021.0, 3.57, 0.0, 0.0]]  
27   [[-7.0, -5.0, 1022.0, 5.36, 1.0, 0.0]]  
28   [[-7.0, -5.0, 1022.0, 6.25, 2.0, 0.0]]

In [380]:
train_label.head()

pollution single_output_vector
24      129.0            [[129.0]]
25      148.0            [[148.0]]
26      159.0            [[159.0]]
27      181.0            [[181.0]]
28      138.0            [[138.0]]

In [381]:
def func(x, maxlen=None):
    '''
    Pad sequences with lists of 0s. Functional lambda programming.
    '''
    zeros_to_add = maxlen - len(x)
    prepended = [np.zeros(7).tolist()]
    y = prepended*zeros_to_add + x
    return y

def func_output(x, maxlen=None):
    '''
    Pad sequences with lists of 0s. Functional lambda programming.
    '''
    zeros_to_add = maxlen - len(x)
    prepended = [np.zeros(1).tolist()]
    y = prepended*zeros_to_add + x
    return y

def balanced_sliding_windows(df, stride=3):
    '''
    Create sliding windows of size [n - 3, n + 3]. Let df=train.
    '''
    seqs = []
    for i, value in df.iterrows():
        if i >= stride and i < len(df) - stride:
            sequences = df.iloc[i-stride:i+stride, -1].sum()
        elif i < stride:
            sequences = df.iloc[i:i+stride, -1].sum()
        else:
            sequences = df.iloc[i-stride:i, -1].sum()
        seqs.append(sequences)
    return pd.Series(seqs)
        
def past_windows(df, stride=3):
    '''
    Create windows of size [n - 3: n]. Let df=train.
    '''
    seqs = []
    for i, value in df.iterrows():
        if i >= stride and i < len(df) - stride:
            sequences = df.iloc[i-stride:i, -1].sum()
        elif i < stride:
            sequences = df.iloc[0:i, -1].sum()
        else:
            sequences = df.iloc[i-stride:i, -1].sum()
        if sequences == 0:
            # Pad empty array with arrays of 0s
            sequences = func([], stride)
        if len(sequences) < stride:
            # Pad with arrays of 0s
            sequences = func(sequences, stride)
        seqs.append(sequences)
    return pd.Series(seqs)

def future_windows(df, stride=3):
    '''
    Create windows of size [n : n+3]. Let df=train.
    '''
    seqs = []
    for i, value in df.iterrows():
        if i >= stride and i < len(df) - stride:
            sequences = df.iloc[i:i+stride, -1].sum()
        elif i < stride:
            sequences = df.iloc[i:i+stride, -1].sum()
        else:
            sequences = df.iloc[i:, -1].sum()
        if type(sequences) == 'int' and sequences == 0:
            sequences = func_output([], stride)
        elif type(sequences) == 'int' and len(sequences) < stride:
            # Pad with arrays of 0s
            print(sequences)
            sequences = func_output(sequences, stride)
        seqs.append(sequences)
    return pd.Series(seqs)

In [382]:
train_features = train_features.reset_index(drop=True)
train_label = train_label.reset_index(drop=True)

In [383]:
input_vec = train_features['single_input_vector']
input_vec = pd.DataFrame(input_vec)

In [384]:
output_vec = train_label['single_output_vector']
output_vec = pd.DataFrame(output_vec)

In [385]:
# Good so far.

In [386]:
#balanced_sliding_windows(input_vec).head()

In [387]:
#past_windows(input_vec).head()

In [388]:
#future_windows(output_vec).head()

In [389]:
#df.head()

In [390]:
# Good so far

In [391]:
def future_windows2(df, stride=3):
    '''
    Create windows of size [n : n+3]. Let df=train.
    '''
    seqs = []
    sequences = None
    for i, value in df.reset_index(drop=True).iterrows():
        if i >= stride and i < len(df) - stride:
            sequences = df.iloc[i:i+stride, -1].sum()
        elif i < stride:
            sequences = df.iloc[i:i+stride, -1].sum()
        else:
            sequences = df.iloc[i:, -1].sum()
        if type(sequences) == 'int' and sequences == 0:
            sequences = func_output([], stride)
        elif i > len(df) - stride:
            # Pad with arrays of 0s
            sequences = func_output(sequences, stride)
        seqs.append(sequences)
    return pd.Series(seqs)

In [392]:
def past_windows2(df, stride=3):
    '''
    Create windows of size [n - 3: n]. Let df=train.
    '''
    seqs = []
    # Bug was in the reset_index, the index started with 24 because the first 24 hours did not have
    # a target value. So the i in iterrows() was taking the 24 + ith index instead of the ith index.
    for i, value in df.reset_index(drop=True).iterrows():
        if i >= stride and i < len(df) - stride:
            sequences = df.iloc[i-stride:i, -1].sum()
        elif i < stride:
            sequences = df.iloc[0:i, -1].sum()
        else:
            sequences = df.iloc[i-stride:i, -1].sum()
        if sequences == 0:
            # Pad empty array with arrays of 0s
            sequences = func([], stride)
        if len(sequences) < stride:
            # Pad with arrays of 0s
            sequences = func(sequences, stride)
        seqs.append(sequences)
    return pd.Series(seqs)

In [393]:
# Debugging past windows
# np.asarray(pd.DataFrame(past_windows2(small_output_vec)).apply(np.asarray, axis=1))

In [394]:
#pd.DataFrame(past_windows2(small_output_vec)).apply(len, axis=1)

In [395]:
train_features['past_sequences'] = past_windows2(input_vec)

In [396]:
train_label['future_sequences'] = future_windows2(output_vec)

In [397]:
small_output_vec = output_vec.iloc[:20,:]

In [398]:
small_input_vec = input_vec.iloc[:20,:]

In [399]:
# Debugging future windows
#future_windows2(small_output_vec)

In [400]:
train_features.head()

dew  temp   press  wnd_spd  snow  rain  \
0  -16  -4.0  1020.0     1.79     0     0   
1  -15  -4.0  1020.0     2.68     0     0   
2  -11  -5.0  1021.0     3.57     0     0   
3   -7  -5.0  1022.0     5.36     1     0   
4   -7  -5.0  1022.0     6.25     2     0   

                       single_input_vector  \
0  [[-16.0, -4.0, 1020.0, 1.79, 0.0, 0.0]]   
1  [[-15.0, -4.0, 1020.0, 2.68, 0.0, 0.0]]   
2  [[-11.0, -5.0, 1021.0, 3.57, 0.0, 0.0]]   
3   [[-7.0, -5.0, 1022.0, 5.36, 1.0, 0.0]]   
4   [[-7.0, -5.0, 1022.0, 6.25, 2.0, 0.0]]   

                                      past_sequences  
0  [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0....  
1  [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0....  
2  [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-16.0, ...  
3  [[-16.0, -4.0, 1020.0, 1.79, 0.0, 0.0], [-15.0...  
4  [[-15.0, -4.0, 1020.0, 2.68, 0.0, 0.0], [-11.0...

In [401]:
# predicting G for now, just a test example
# If your output is multi-dimensional, you need to capture those 
# dimensions in one object
# If your output is a single dimension, this step may be unnecessary
def set_output_featureset(df):
    '''
    Let df=train_label and prepare output vector
    '''
    df['output_vector'] = df['future_sequences']
    return df

def set_input_featureset(df):
    '''
    Let df=train_features and prepare input vector
    '''
    df['input_vector'] = df['past_sequences']
    return df

In [402]:
train_label = set_output_featureset(train_label)

In [403]:
train_label.head()

pollution single_output_vector             future_sequences  \
0      129.0            [[129.0]]  [[129.0], [148.0], [159.0]]   
1      148.0            [[148.0]]  [[148.0], [159.0], [181.0]]   
2      159.0            [[159.0]]  [[159.0], [181.0], [138.0]]   
3      181.0            [[181.0]]  [[181.0], [138.0], [109.0]]   
4      138.0            [[138.0]]  [[138.0], [109.0], [105.0]]   

                 output_vector  
0  [[129.0], [148.0], [159.0]]  
1  [[148.0], [159.0], [181.0]]  
2  [[159.0], [181.0], [138.0]]  
3  [[181.0], [138.0], [109.0]]  
4  [[138.0], [109.0], [105.0]]

In [404]:
train_features = set_input_featureset(train_features)

In [405]:
train_features.head()

dew  temp   press  wnd_spd  snow  rain  \
0  -16  -4.0  1020.0     1.79     0     0   
1  -15  -4.0  1020.0     2.68     0     0   
2  -11  -5.0  1021.0     3.57     0     0   
3   -7  -5.0  1022.0     5.36     1     0   
4   -7  -5.0  1022.0     6.25     2     0   

                       single_input_vector  \
0  [[-16.0, -4.0, 1020.0, 1.79, 0.0, 0.0]]   
1  [[-15.0, -4.0, 1020.0, 2.68, 0.0, 0.0]]   
2  [[-11.0, -5.0, 1021.0, 3.57, 0.0, 0.0]]   
3   [[-7.0, -5.0, 1022.0, 5.36, 1.0, 0.0]]   
4   [[-7.0, -5.0, 1022.0, 6.25, 2.0, 0.0]]   

                                      past_sequences  \
0  [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0....   
1  [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0....   
2  [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-16.0, ...   
3  [[-16.0, -4.0, 1020.0, 1.79, 0.0, 0.0], [-15.0...   
4  [[-15.0, -4.0, 1020.0, 2.68, 0.0, 0.0], [-11.0...   

                                        input_vector  
0  [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0....  
1  [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0....  
2  [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-16.0, ...  
3  [[-16.0, -4.0, 1020.0, 1.79, 0.0, 0.0], [-15.0...  
4  [[-15.0, -4.0, 1020.0, 2.68, 0.0, 0.0], [-11.0...

### Split Train into Train and Dev

In [406]:
# train_features, dev_features, train_label, dev_label = train_test_split(train_features, train_label,
#                                                                           test_size=.20, 
#                                                                           random_state=789)

In [407]:
# Extract your training data
#X_train_init = np.asarray(train.past_sequences)
X_train_init = np.asarray(train_features.past_sequences)
stride_length = 3
# Training data for LSTM should be in the form of a 3D tuple:
#   (# of samples, timesteps, input_dim)
# Note that the input data that comes out of the dataframe 
# will not make a 3D array. It makes an array of arrays, 
# which is not the same thing.
# So far X_train_init is an array of arrays.

# Convert to 3D vector usng hstack and reshape
# horizontal stack = hstack, essentially removes the outer array 
#  encapsulation.

# reshape into (# of records, total_timesteps, input_dim)
# The reshape essentially reshaped the inner list into an 11 by 6 matrix,
#  or a max_sequence_length (rows) by input_dim (col) matrix
X_train = np.hstack(X_train_init).reshape(len(train_features), 
                                          stride_length,
                                         len(df_features.columns) + len(df_label.columns))
y_train_init = np.asarray(train_label.output_vector)
y_train = np.hstack(y_train_init).reshape(len(train_features), 
                                          stride_length,
                                          len(output_vec.columns))

ValueError: all the input arrays must have same number of dimensions

In [411]:
pd.DataFrame(np.asarray(train_features.input_vector)).apply(np.asarray, axis=1).apply(np.hstack, axis=1)

Exception: Data must be 1-dimensional

In [344]:
np.asarray(X_train_init)

array([ list([[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]),
       list([[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-16.0, -4.0, 1020.0, 1.79, 0.0, 0.0]]),
       list([[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-16.0, -4.0, 1020.0, 1.79, 0.0, 0.0], [-15.0, -4.0, 1020.0, 2.6800000000000002, 0.0, 0.0]]),
       ...,
       list([[-19.0, 7.0, 1013.0, 114.87, 0.0, 0.0], [-21.0, 7.0, 1014.0, 119.79000000000001, 0.0, 0.0], [-21.0, 7.0, 1014.0, 125.59999999999999, 0.0, 0.0]]),
       list([[-21.0, 7.0, 1014.0, 119.79000000000001, 0.0, 0.0], [-21.0, 7.0, 1014.0, 125.59999999999999, 0.0, 0.0], [-21.0, 6.0, 1014.0, 130.52000000000001, 0.0, 0.0]]),
       list([[-21.0, 7.0, 1014.0, 125.59999999999999, 0.0, 0.0], [-21.0, 6.0, 1014.0, 130.52000000000001, 0.0, 0.0], [-20.0, 7.0, 1014.0, 137.66999999999999, 0.0, 0.0]])], dtype=object)

In [342]:
pd.DataFrame(X_train_init)

0
0      [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0....
1      [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0....
2      [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-16.0, ...
3      [[-16.0, -4.0, 1020.0, 1.79, 0.0, 0.0], [-15.0...
4      [[-15.0, -4.0, 1020.0, 2.68, 0.0, 0.0], [-11.0...
5      [[-11.0, -5.0, 1021.0, 3.57, 0.0, 0.0], [-7.0,...
6      [[-7.0, -5.0, 1022.0, 5.36, 1.0, 0.0], [-7.0, ...
7      [[-7.0, -5.0, 1022.0, 6.25, 2.0, 0.0], [-7.0, ...
8      [[-7.0, -6.0, 1022.0, 7.14, 3.0, 0.0], [-7.0, ...
9      [[-7.0, -6.0, 1023.0, 8.93, 4.0, 0.0], [-7.0, ...
10     [[-7.0, -5.0, 1024.0, 10.72, 0.0, 0.0], [-8.0,...
11     [[-8.0, -6.0, 1024.0, 12.51, 0.0, 0.0], [-7.0,...
12     [[-7.0, -5.0, 1025.0, 14.3, 0.0, 0.0], [-7.0, ...
13     [[-7.0, -5.0, 1026.0, 17.43, 1.0, 0.0], [-8.0,...
14     [[-8.0, -5.0, 1026.0, 20.56, 0.0, 0.0], [-8.0,...
15     [[-8.0, -5.0, 1026.0, 23.69, 0.0, 0.0], [-8.0,...
16     [[-8.0, -5.0, 1025.0, 27.71, 0.0, 0.0], [-9.0,...
17     [[-9.0, -5.0, 1025.0, 31.73, 0.0, 0.0], [-9.0,...
18     [[-9.0, -5.0, 1025.0, 35.75, 0.0, 0.0], [-9.0,...
19     [[-9.0, -5.0, 1026.0, 37.54, 0.0, 0.0], [-8.0,...
20     [[-8.0, -5.0, 1027.0, 39.33, 0.0, 0.0], [-8.0,...
21     [[-8.0, -5.0, 1027.0, 42.46, 0.0, 0.0], [-8.0,...
22     [[-8.0, -5.0, 1028.0, 44.25, 0.0, 0.0], [-7.0,...
23     [[-7.0, -5.0, 1028.0, 46.04, 0.0, 0.0], [-7.0,...
24     [[-7.0, -5.0, 1027.0, 49.17, 1.0, 0.0], [-8.0,...
25     [[-8.0, -6.0, 1028.0, 52.3, 2.0, 0.0], [-8.0, ...
26     [[-8.0, -6.0, 1027.0, 55.43, 3.0, 0.0], [-7.0,...
27     [[-7.0, -6.0, 1027.0, 58.56, 4.0, 0.0], [-8.0,...
28     [[-8.0, -6.0, 1026.0, 61.69, 5.0, 0.0], [-8.0,...
29     [[-8.0, -7.0, 1026.0, 65.71, 6.0, 0.0], [-8.0,...
...                                                  ...
35011  [[-23.0, 7.0, 1014.0, 120.68, 0.0, 0.0], [-22....
35012  [[-22.0, 6.0, 1015.0, 126.49, 0.0, 0.0], [-22....
35013  [[-22.0, 5.0, 1015.0, 130.51, 0.0, 0.0], [-22....
35014  [[-22.0, 6.0, 1015.0, 132.3, 0.0, 0.0], [-21.0...
35015  [[-21.0, 5.0, 1015.0, 3.13, 0.0, 0.0], [-20.0,...
35016  [[-20.0, 3.0, 1015.0, 1.79, 0.0, 0.0], [-20.0,...
35017  [[-20.0, 3.0, 1014.0, 3.13, 0.0, 0.0], [-21.0,...
35018  [[-21.0, 4.0, 1014.0, 4.92, 0.0, 0.0], [-18.0,...
35019  [[-18.0, -2.0, 1013.0, 0.89, 0.0, 0.0], [-20.0...
35020  [[-20.0, 1.0, 1013.0, 1.79, 0.0, 0.0], [-18.0,...
35021  [[-18.0, -2.0, 1012.0, 0.89, 0.0, 0.0], [-16.0...
35022  [[-16.0, -5.0, 1012.0, 1.79, 0.0, 0.0], [-16.0...
35023  [[-16.0, -2.0, 1011.0, 3.58, 0.0, 0.0], [-15.0...
35024  [[-15.0, -5.0, 1011.0, 1.79, 0.0, 0.0], [-19.0...
35025  [[-19.0, 2.0, 1010.0, 3.13, 0.0, 0.0], [-20.0,...
35026  [[-20.0, 5.0, 1011.0, 10.28, 0.0, 0.0], [-19.0...
35027  [[-19.0, 5.0, 1012.0, 17.43, 0.0, 0.0], [-19.0...
35028  [[-19.0, 6.0, 1013.0, 27.26, 0.0, 0.0], [-18.0...
35029  [[-18.0, 7.0, 1013.0, 37.09, 0.0, 0.0], [-19.0...
35030  [[-19.0, 7.0, 1013.0, 46.03, 0.0, 0.0], [-18.0...
35031  [[-18.0, 9.0, 1012.0, 57.21, 0.0, 0.0], [-18.0...
35032  [[-18.0, 9.0, 1011.0, 65.26, 0.0, 0.0], [-19.0...
35033  [[-19.0, 9.0, 1011.0, 73.31, 0.0, 0.0], [-18.0...
35034  [[-18.0, 9.0, 1012.0, 83.14, 0.0, 0.0], [-18.0...
35035  [[-18.0, 9.0, 1012.0, 92.97, 0.0, 0.0], [-19.0...
35036  [[-19.0, 8.0, 1012.0, 102.8, 0.0, 0.0], [-19.0...
35037  [[-19.0, 8.0, 1013.0, 109.95, 0.0, 0.0], [-19....
35038  [[-19.0, 7.0, 1013.0, 114.87, 0.0, 0.0], [-21....
35039  [[-21.0, 7.0, 1014.0, 119.79, 0.0, 0.0], [-21....
35040  [[-21.0, 7.0, 1014.0, 125.6, 0.0, 0.0], [-21.0...

[35041 rows x 1 columns]

In [291]:
np.hstack(X_train_init)

ValueError: all the input arrays must have same number of dimensions

In [92]:
#np.hstack(y_train_init)

In [93]:
len(train), len(df.columns)

(43800, 9)

In [283]:
# #Debugging
#train[train.output_vector.apply(lambda x: True if len(x) != 3 else False)]

TypeError: object of type 'int' has no len()

In [347]:
train_features[train_features.input_vector.apply(lambda x: True if len(x) != 3 else False)]

Empty DataFrame
Columns: [dew, temp, press, wnd_spd, snow, rain, single_input_vector, past_sequences, input_vector]
Index: []

In [95]:
print(X_train.shape)
print(y_train.shape)

(43800, 3, 7)
(43800, 3, 1)


In [96]:
# Get your input dimensions
# Input length is the length for one input sequence 
#  (i.e. the number of rows for your sample, which is
#     the max_sequence_length by construction)
input_length = X_train.shape[1]
# Input dim is the number of dimensions in one input vector 
#  (i.e. number of input columns)
input_dim = X_train.shape[2]
output_dim = len(y_train[0])

In [97]:
input_length

3

In [101]:
from keras.layers import CuDNNLSTM
# Room to build a GPU-powered network using CuDNNLSTM
model_GPU = Sequential()
model_GPU.add(CuDNNLSTM(100, input_shape=(input_length, input_dim), return_sequences=True))
model_GPU.add(Dropout(.20))
model_GPU.add(CuDNNLSTM(50, input_shape=(input_length, input_dim), return_sequences=True))
model_GPU.add(Dropout(.20))
model_GPU.add(TimeDistributed(Dense(1, activation='softmax')))



ImportError: cannot import name 'CuDNNLSTM'

In [141]:
model_GPU.compile(loss='mse',
             optimizer='adam',
             metrics=['accuracy'])

In [143]:
# Set batch_size to 30 to show that it doesn't have to be a factor 
# or multiple of your sample size
history = model_GPU.fit(X_train, y_train,
                   batch_size = 24*5, epochs=10,
                   verbose = 1)

Epoch 1/10
35040/35040 [==============================] - 2s 66us/step - loss: 2.0061 - acc: 0.0000e+00
Epoch 2/10
35040/35040 [==============================] - 2s 66us/step - loss: 2.0061 - acc: 0.0000e+00
Epoch 3/10
35040/35040 [==============================] - 2s 66us/step - loss: 2.0061 - acc: 0.0000e+00
Epoch 4/10
35040/35040 [==============================] - 2s 66us/step - loss: 2.0061 - acc: 0.0000e+00
Epoch 5/10
35040/35040 [==============================] - 2s 66us/step - loss: 2.0061 - acc: 0.0000e+00
Epoch 6/10
35040/35040 [==============================] - 2s 67us/step - loss: 2.0061 - acc: 0.0000e+00
Epoch 7/10
35040/35040 [==============================] - 2s 67us/step - loss: 2.0061 - acc: 0.0000e+00
Epoch 8/10
35040/35040 [==============================] - 2s 66us/step - loss: 2.0061 - acc: 0.0000e+00
Epoch 9/10
35040/35040 [==============================] - 2s 66us/step - loss: 2.0061 - acc: 0.0000e+00
Epoch 10/10
35040/35040 [==============================] - 2s 67

In [ ]:
# Reduced from 2 hours per epoch in a CPU to 2 minutes per epoch in the GPU

In [87]:
from keras.models import Model, Sequential
from keras.layers import LSTM, Dense, TimeDistributed, Dropout

# Initialize the model
model_simple = Sequential()
# 100 is the units, which is the size of the hidden vector at any given state
# input_length is the stride_length, which is equivalent to the number of cells
model_simple.add(LSTM(100, input_shape=(input_length, input_dim), return_sequences=True))
#model_simple.add(LSTM(50, input_shape=(input_length, input_dim), return_sequences=True))
model_simple.add(TimeDistributed(Dense(1, activation='linear')))


# Below is a very large, 6-layer network (Takes a lot of time to run)
model = Sequential()
# arbitrarily picked the output dim to be 100
model.add(LSTM(2048, input_shape=(input_length, input_dim), return_sequences=True))
# The max output value is > 1 so used relu as final activation
# 2048 is the output dimension in the layer
# Input to this layer will have a shape (None, 3, 50) where 3 = stride_length, None = batch_size,
#   and 2048 = output layer size
# Add dropout probability
model.add(Dropout(0.2))
# Another LSTM layer
model.add(LSTM(1024, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.2))
# Output dimension of 7, since I have a feature length of 7 (pollution + weather stuff)
#model.add(Dense(7, activation='softmax'))
# Output dimension of 1, since I have a feature length of 1 (pollution is only output feature)
model.add(TimeDistributed(Dense(1, activation='linear')))
#model.add(Dense(output_dim, activation='relu'))

# model.compile(loss='mean_squared_error',
#              optimizer='rmsprop',
#              metrics=['accuracy'])

In [88]:
model_simple.compile(loss='mse',
             optimizer='adam',
             metrics=['accuracy'])

In [89]:
# Set batch_size to 30 to show that it doesn't have to be a factor 
# or multiple of your sample size
history = model_simple.fit(X_train, y_train,
                   batch_size = 20, epochs=5,
                   verbose = 1)

Epoch 1/5
35040/35040 [==============================] - 17s 488us/step - loss: 13368.9472 - acc: 0.0070
Epoch 2/5
35040/35040 [==============================] - 14s 405us/step - loss: 9420.4976 - acc: 0.0052
Epoch 3/5
35040/35040 [==============================] - 14s 405us/step - loss: 8671.4822 - acc: 0.0042
Epoch 4/5
35040/35040 [==============================] - 14s 405us/step - loss: 8628.8452 - acc: 0.0039
Epoch 5/5
35040/35040 [==============================] - 14s 406us/step - loss: 8627.3278 - acc: 0.0041


In [ ]:
# Took 1.5min on CPU --> On GPU it took 40 seconds

In [43]:
model.predict(X_train)

array([[[   9.75070953],
        [  36.31097412],
        [ 104.66318512]],

       [[  82.32813263],
        [  97.49900055],
        [  99.82424927]],

       [[  82.32813263],
        [  97.49900055],
        [  99.82424927]],

       ..., 
       [[  82.32813263],
        [  97.49900055],
        [  99.82424927]],

       [[  82.32813263],
        [  97.49900055],
        [  99.82424927]],

       [[  82.32813263],
        [  97.49900055],
        [  99.82424927]]], dtype=float32)

In [44]:
y_train

array([[[  11.],
        [  27.],
        [  85.]],

       [[  15.],
        [  28.],
        [  21.]],

       [[ 143.],
        [  79.],
        [ 230.]],

       ..., 
       [[  53.],
        [ 259.],
        [  93.]],

       [[  85.],
        [ 118.],
        [  11.]],

       [[ 126.],
        [ 126.],
        [ 126.]]])